# Phase 1 : 베이스라인(Baseline) 성능 측정

#### [Phase 1 : 베이스라인 성능 측정 결과 분석]

Phase 1의 베이스라인 측정 결과, 파인튜닝된 모델(SFT, PPO)이 학습 데이터의 품질 문제로 인해 심각한 '모델 붕괴(Model Collapse)' 현상을 겪고 있음을 확인했습니다.

---

#### [모델별 결과 분석]

**1. Pre-trained KoGPT-2 (원본)**
- 작동 방식 : 이 모델은 지시를 따르기보다는, 주어진 단어 다음에 통계적으로 가장 올 확률이 높은 단어를 예측하는 기본적인 언어 모델의 특성을 그대로 보여줍니다. "대한민국의 수도는 어디인가요?"라는 질문에 "서울?"이라는 단어를 반복하는 것처럼, 질문을 이해하고 답변하는 것이 아니라 단순히 다음 단어를 생성할 뿐입니다.
- 평가 : 대화형 모델로서 기능하지는 않지만, 앞으로 진행될 모든 파인튜닝의 성능 비교 기준점으로서 중요한 의미를 가집니다.

**2. SFT & PPO 파인튜닝 모델**

SFT와 PPO 모델은 정도의 차이는 있지만, 공통적으로 심각한 성능 저하를 보였습니다. 이는 파인튜닝이 항상 성능 향상으로 이어지는 것이 아니며, 학습 데이터의 품질이 매우 중요하다는 것을 증명합니다. 주요 문제점은 다음과 같습니다.

- **언어 붕괴 및 오염 (Language Collapse & Contamination)**
    - 현상 : 한국어 문장 중간에 의미를 알 수 없는 영어(inhery for the information...), 일본어(シンンン...), 한자(具羅也也也...)가 섞여 출력되었습니다. 이는 모델이 정상적인 언어 생성 능력을 상실했음을 의미합니다.
- 원인 : 학습 데이터셋에 정제되지 않은 노이즈(웹 스크래핑 찌꺼기, 다른 언어 등)가 포함되었을 가능성이 매우 높습니다.

- **의미 없는 반복 (Meaningless Repetition)**
    - 현상 : "세종대왕의 가장 큰 업적은?"이라는 질문에 "『삼국사기』『삼국사기』..."와 같이, 질문과 전혀 관련 없는 단어를 끝없이 반복했습니다.
    - 원인 : 모델이 특정 토큰에 과도하게 집중하여 빠져나오지 못하는 현상으로, 학습이 불안정할 때 발생합니다.

- **지시 불이행 (Instruction Following Failure)**
    - 현상 : "파이썬으로 간단한 웹 서버 만드는 코드 좀 짜줘"와 같은 구체적인 요구에 코드를 생성하지 않고, "어떤 서버에 어떤 프로그램을 쓰시는지"라고 되묻는 등, 사용자의 지시를 전혀 이해하거나 수행하지 못했습니다.
    - 원인 : 모델이 대화의 '형식'(질문-답변)만 학습했을 뿐, 내용의 '의미'를 이해하는 학습이 부족했기 때문입니다.

- **페르소나 과잉 학습 (Persona Overfitting)**
    - 현상 : "오늘 하루 어땠어?" 같은 간단한 질문에 "저는 AI 챗봇이라 주어진 임무만 열심히 합니다"와 같이, 지나치게 정형화된 답변을 내놓았습니다.
    - 원인 : 학습 데이터에 "저는 AI입니다"와 같은 특정 패턴의 답변이 과도하게 포함되어, 모델이 해당 답변만 집중적으로 학습했기 때문입니다.

---

#### [결론 및 다음 단계]

Phase 1 베이스라인 측정을 통해, 현재의 KoChatGPT 모델은 파인튜닝을 거쳐도 실용적인 대화 능력을 갖추지 못했음이 명확해졌습니다. 특히 SFT와 PPO 단계에서 성능이 급격히 저하되는 현상은 모델 자체의 문제라기보다 학습에 사용된 데이터셋(kochatgpt_1_SFT.jsonl 등)에 심각한 품질 문제가 있음을 강력하게 시사합니다.

따라서 모델의 성능을 향상시키기 위한 다음 단계로는 '데이터셋 정밀 분석 및 정제'를 진행하고자 합니다. 품질이 낮은 데이터, 노이즈, 불필요한 반복 패턴 등을 제거하고 양질의 데이터셋을 구축하여 재학습시키는 것이 모델을 정상화하기 위한 핵심적인 해결책이 될 것 같습니다.

## ✅ Step 1 : 환경 설정 및 테스트 프롬프트셋 생성

가장 먼저 필요한 라이브러리를 불러오고, 모델의 성능을 일관되게 평가하기 위한 질문 목록을 만듭니다.

In [1]:
!pip install datasets evaluate rouge_score nltk accelerate transformers[torch]

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

# 사용할 모델과 토크나이저를 미리 정의합니다.
# 1. 원본 KoGPT-2 모델
PRETRAINED_MODEL_NAME = 'skt/kogpt2-base-v2'

# 2. SFT 파인튜닝된 모델이 저장된 경로
SFT_MODEL_PATH = './models/output_1_SFT' 

# 3. PPO 파인튜닝된 모델이 저장된 경로
PPO_MODEL_PATH = './models/output_3_PPO'

# 일관된 평가를 위한 테스트 프롬프트 목록
test_prompts = [
    "대한민국의 수도는 어디인가요?",
    "세종대왕의 가장 큰 업적은 무엇인가요?",
    "임진왜란이 언제 일어났는지 알려줘.",
    "요즘 피곤한데 힘이 나는 말을 해줘.",
    "AI가 세상을 어떻게 바꿀 수 있을까?",
    "저녁 메뉴 추천해줘.",
    "파이썬으로 간단한 웹 서버 만드는 코드 좀 짜줘.",
    "오늘 하루 어땠어?",
    "바다는 왜 파란색이야?",
    "행복이란 무엇이라고 생각해?"
]

# 결과를 저장할 리스트
results = []

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## ✅ Step 2 : 모델별 결과물 생성 및 기록

이제 위에서 정의한 test_prompts 목록을 가지고 각 모델(원본, SFT, PPO)의 답변을 생성합니다.

In [3]:
print("="*50)
print("1. Generating from Pre-trained KoGPT-2 Model...")
print("="*50)

# 원본 모델 및 토크나이저 로드
base_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(PRETRAINED_MODEL_NAME).to(device)

# 각 프롬프트에 대해 답변 생성
for prompt in test_prompts:
    inputs = base_tokenizer.encode(prompt, return_tensors='pt').to(device)

    # generate 함수에 pad_token_id를 명시적으로 설정해주는 것이 안정적
    if base_tokenizer.pad_token_id is None:
        base_tokenizer.pad_token_id = base_tokenizer.eos_token_id
    
    # 답변 생성
    # do_sample=True: 좀 더 창의적인 답변 생성
    # top_k=50, temperature=0.7: 너무 벗어나지 않으면서도 다양한 표현을 사용하도록 설정
    output_sequences = base_model.generate(
        input_ids=inputs,
        max_length=100,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        temperature=0.7,
        eos_token_id=base_tokenizer.eos_token_id,
        pad_token_id=base_tokenizer.pad_token_id
    )
    
    # 생성된 답변 디코딩
    generated_text = base_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    
    # 결과 저장
    results.append({
        'prompt': prompt,
        'model_type': 'Pre-trained KoGPT-2',
        'response': generated_text
    })
    
    print(f"Prompt: {prompt}")
    print(f"Response: {generated_text}\n" + "-"*30)

# 메모리 관리를 위해 모델 삭제
del base_model
torch.cuda.empty_cache()

1. Generating from Pre-trained KoGPT-2 Model...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Prompt: 대한민국의 수도는 어디인가요?
Response: 대한민국의 수도는 어디인가요?" "서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울?"
"서울
------------------------------
Prompt: 세종대왕의 가장 큰 업적은 무엇인가요?
Response: 세종대왕의 가장 큰 업적은 무엇인가요?"
"그것은 바로 신하의 덕이 아니었는가?"
"전하, 지금 이 자리에서 말씀드리겠습니다."
"그것은 바로 신하의 덕이 아니었는가?"
"그것은 바로 신하의 덕이 아니었는가."
"그것은 바로 신하의 덕이 아니었는가?"
"그것은 바로 신하의 덕이었다."
"그것은 바로 신하의 덕이 아니었는가?"
"그것은 바로 신하의 덕이 아니겠는가?
------------------------------
Prompt: 임진왜란이 언제 일어났는지 알려줘.
Response: 임진왜란이 언제 일어났는지 알려줘. 네. 네. 그렇죠.
예. 그니까 이게 지금 뭐 어떻게 보면 인제 지금 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이~ 이
------------------------------
Prompt: 요즘 피곤한데 힘이 나는 말을 해줘.
Response: 요즘 피곤한데 힘이 나는 말을 해줘."
"저런 말도 해줘."
"그럼, 그게 무슨 소리야?"
"내가 어떻게 저렇게 많은 말을 해야 할 수 있는 거야?"
"아뇨, 제가 조금 전에 말했잖아요. 저는 이미 세상을 떠난 사람이기 때문에."
"무슨 말이에요, 어젯밤까지 나한테 얘기하지도 않는 건가요?"
"그런데 왜 저렇게 말을 해요."
"여전히
------------------------------


## 📝 2-2. SFT 적용 모델 결과 생성

이제 SFT 학습을 마친 모델을 불러와 동일한 작업을 반복합니다.

In [4]:
print("="*50)
print("2. Generating from SFT Fine-tuned Model...")
print("="*50)

# SFT 모델 및 토크나이저 로드
# 토크나이저는 원본 KoGPT-2 경로에서 불러옵니다.
sft_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
# 모델 가중치는 파인튜닝된 SFT 경로에서 그대로 불러옵니다.
sft_model = AutoModelForCausalLM.from_pretrained(SFT_MODEL_PATH).to(device)

# 각 프롬프트에 대해 답변 생성
for prompt in test_prompts:
    inputs = sft_tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    # generate 함수에 pad_token_id를 명시적으로 설정해주는 것이 안정적
    if sft_tokenizer.pad_token_id is None:
        sft_tokenizer.pad_token_id = sft_tokenizer.eos_token_id
        
    output_sequences = sft_model.generate(
        input_ids=inputs,
        max_length=100,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        temperature=0.7,
        eos_token_id=sft_tokenizer.eos_token_id,
        pad_token_id=sft_tokenizer.pad_token_id
    )
    
    generated_text = sft_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    
    results.append({
        'prompt': prompt,
        'model_type': 'SFT Model',
        'response': generated_text
    })
    
    print(f"Prompt: {prompt}")
    print(f"Response: {generated_text}\n" + "-"*30)
    
# 메모리 관리를 위해 모델 삭제
del sft_model
torch.cuda.empty_cache()

2. Generating from SFT Fine-tuned Model...
Prompt: 대한민국의 수도는 어디인가요?
Response: 대한민국의 수도는 어디인가요?
------------------------------
Prompt: 세종대왕의 가장 큰 업적은 무엇인가요?
Response: 세종대왕의 가장 큰 업적은 무엇인가요?\n\n<세종대왕의 가장 큰 업적은 조선 시대(세종대왕의 가장 큰 업적 중 하나였습니다. 조선 시대에 그는 세종대왕의 가장 큰 업적 중 하나로 꼽히는 고구려 고구려성벽, 고구려 역사문화유산 등재를 추진하여 그 성과를 이룩하였습니다. 이 외에도, 그는 세종대왕의 가장 큰 업적은 고구려 문화 콘텐츠 개발과 고구려 역사학 연구 등을 통해 고구려 문화 발전을 이루었습니다. 이 외에도, 그는 고구려 역사문화를 연구하며 다양한 역사 관련
------------------------------
Prompt: 임진왜란이 언제 일어났는지 알려줘.
Response: 임진왜란이 언제 일어났는지 알려줘. 예를 들면 "우리나라에서 일어난 역사적인 사건"을 제시해 주신다면 구체적으로 답변을 드릴 수 있을 것 같습니다.者, physical context or universal context or universal context or universal context or universal context or universal context or universal con
------------------------------
Prompt: 요즘 피곤한데 힘이 나는 말을 해줘.
Response: 요즘 피곤한데 힘이 나는 말을 해줘. 그러면 안 되는 것은 아닌 것 같습니다. 상황을 설명해주시면 더 자세한 도움을 드릴 수 있을 것 같습니다.意: \n\n1. \n2. \n3. \n4. \n5. \n6. \n7. \n7. \n8. \n9. \n9. \n10. \n\n10. \n10
------------------------------
Prompt: AI가 세상을 어떻게

## 📝 2-3. 최종 RM/PPO 적용 모델 결과 생성

마지막으로 PPO 학습까지 완료된 최종 모델의 답변을 생성합니다.

In [5]:
print("="*50)
print("3. Generating from PPO Fine-tuned Model...")
print("="*50)

# PPO 모델 및 토크나이저 로드
# 토크나이저는 원본 KoGPT-2 경로에서 불러옵니다.
ppo_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
# 모델 가중치는 파인튜닝된 PPO 경로에서 그대로 불러옵니다.
ppo_model = AutoModelForCausalLM.from_pretrained(PPO_MODEL_PATH).to(device)

# 각 프롬프트에 대해 답변 생성
for prompt in test_prompts:
    inputs = ppo_tokenizer.encode(prompt, return_tensors='pt').to(device)

    # generate 함수에 pad_token_id를 명시적으로 설정해주는 것이 안정적
    if ppo_tokenizer.pad_token_id is None:
        ppo_tokenizer.pad_token_id = ppo_tokenizer.eos_token_id

    output_sequences = ppo_model.generate(
        input_ids=inputs,
        max_length=100,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        temperature=0.7,
        eos_token_id=ppo_tokenizer.eos_token_id,
        pad_token_id=ppo_tokenizer.pad_token_id
    )

    generated_text = ppo_tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    results.append({
        'prompt': prompt,
        'model_type': 'PPO Model',
        'response': generated_text
    })

    print(f"Prompt: {prompt}")
    print(f"Response: {generated_text}\n" + "-"*30)

# 메모리 관리를 위해 모델 삭제
del ppo_model
torch.cuda.empty_cache()

3. Generating from PPO Fine-tuned Model...
Prompt: 대한민국의 수도는 어디인가요?
Response: 대한민국의 수도는 어디인가요? 자세한 정보를 알려주시면 더 정확한 답변을 드릴 수 있습니다. 具羅也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也也
------------------------------
Prompt: 세종대왕의 가장 큰 업적은 무엇인가요?
Response: 세종대왕의 가장 큰 업적은 무엇인가요?\n\n『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』『삼국사기』』『삼국
------------------------------
Prompt: 임진왜란이 언제 일어났는지 알려줘.
Response: 임진왜란이 언제 일어났는지 알려줘. inhery for the information of the individer which unn\nIt's personal physical which the information of the information of the information of the information of the information of the information of the Information of the information of the information of the information of the information of
------------------------------
Prompt: 요즘 피곤한데 힘이 나는 말을 해줘.
Response: 요즘 피곤한데 힘이 나는 말을 해줘. 뭐야? 너는 왜 그런 말을 하는지 모르겠어. 왜 그런 말을 하는지 구체적으로 이야기해봐봐. 괜찮아. 왜 그런 말을 하는지 생각해보세. 그리고 그 말이 어떤 의미를 가지는지 더 자세히 설명해보는 것도 좋은 방법일 거야. 그리고 함께 

## ✅ Step 3 : 결과 취합 및 파일로 저장

이제 results 리스트에 저장된 모든 답변을 보기 쉽게 하나의 데이터프레임으로 만들고, 나중 분석을 위해 CSV 파일로 저장합니다.

In [6]:
# 결과를 데이터프레임으로 변환
df_results = pd.DataFrame(results)

# 보기 쉽게 피벗 테이블 형태로 변환
df_pivot = df_results.pivot(index='prompt', columns='model_type', values='response')

# 컬럼 순서 정리
df_pivot = df_pivot[['Pre-trained KoGPT-2', 'SFT Model', 'PPO Model']]

# 결과 출력
print("="*50)
print("Baseline Performance Measurement Results")
print("="*50)
print(df_pivot)

# 결과를 CSV 파일로 저장
baseline_results_path = './baseline_results.csv'
df_pivot.to_csv(baseline_results_path, encoding='utf-8-sig')

print(f"\n Baseline results have been saved to '{baseline_results_path}'")

Baseline Performance Measurement Results
model_type                                                 Pre-trained KoGPT-2  \
prompt                                                                           
AI가 세상을 어떻게 바꿀 수 있을까?        AI가 세상을 어떻게 바꿀 수 있을까?"라는 질문에 "그런 질문은 처음 봤다. 하지...   
대한민국의 수도는 어디인가요?             대한민국의 수도는 어디인가요?" "서울?"\n"서울?"\n"서울?"\n"서울?"\n...   
바다는 왜 파란색이야?                 바다는 왜 파란색이야?"\n"그래. 이건 무슨 색이야?"\n"이건 초록색이지. 파란...   
세종대왕의 가장 큰 업적은 무엇인가요?        세종대왕의 가장 큰 업적은 무엇인가요?"\n"그것은 바로 신하의 덕이 아니었는가?"...   
오늘 하루 어땠어?                   오늘 하루 어땠어?"\n"아~ 그랬어? 난 어떡해?"\n"뭐야. 엄마랑 아빠랑 같이...   
요즘 피곤한데 힘이 나는 말을 해줘.         요즘 피곤한데 힘이 나는 말을 해줘."\n"저런 말도 해줘."\n"그럼, 그게 무슨...   
임진왜란이 언제 일어났는지 알려줘.          임진왜란이 언제 일어났는지 알려줘. 네. 네. 그렇죠.\n예. 그니까 이게 지금 뭐...   
저녁 메뉴 추천해줘.                  저녁 메뉴 추천해줘. 아~ 이메뉴가 정말 짱이야. 그래서 두 가지 메뉴를 주문했어....   
파이썬으로 간단한 웹 서버 만드는 코드 좀 짜줘.  파이썬으로 간단한 웹 서버 만드는 코드 좀 짜줘. 네. 코드 좀 짜줘."\n"아, ...   
행복이란 무엇이라고 생각해?              행복이란 무엇이라고 생각해?"\n"그런 것 같으면.